In [2]:
pdbbind = 'pdbbind_2020/'

pdb_ids = []
with open('pdbbind_2020/index/INDEX_structure.2020') as file:
    for line in file:
        if line[0] != '#':
            line = line.strip().split()
            pdb_ids.append(line[0])


len(pdb_ids)

19443

In [11]:
# random subset to speed up calculations
import random

random.seed(42069)

pdb_ids = random.sample(pdb_ids, 10)
pdb_ids

['3ivq',
 '5zde',
 '4f4p',
 '4d63',
 '2ggd',
 '4fxf',
 '5w9g',
 '4m5k',
 '2bv4',
 '2pyi']

In [30]:
%load_ext autoreload
%autoreload 2

from tqdm import tqdm
from rdkit import Chem
from rdkit.Chem import AllChem

from clean_pdb import Cleaner

cleaner = Cleaner()

errors = {}
clean_pdbs = {}
mols = {}

for pdb in tqdm(pdb_ids):
    path = pdbbind + pdb + '/' + pdb 
    pdbfile, fastaseq, nres, flags = cleaner.apply(path + '_protein.pdb')
    if flags['Successful']:
        clean_pdbs[pdb] = pdbfile
    else:
        errors[pdb] = flags
    mols[pdb] = Chem.MolFromMol2File(path + "_ligand.mol2", sanitize=True, removeHs=True, cleanupSubstructures=True)

print("Errors:", len(errors))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


100%|██████████| 10/10 [00:01<00:00,  8.55it/s]

Errors: 0


In [42]:
print(clean_pdbs['3ivq'])

ATOM      1  N   GLY A   1      -3.869 -11.444  14.343  1.00 53.05           N
ATOM      2  HN1 GLY A   1      -3.716 -12.468  14.436  1.00  0.00           H
ATOM      3  HN2 GLY A   1      -4.019 -11.028  15.284  1.00  0.00           H
ATOM      4  HN3 GLY A   1      -4.704 -11.270  13.749  1.00  0.00           H
ATOM      5  CA  GLY A   1      -2.673 -10.820  13.711  1.00 52.74           C
ATOM      6  HA1 GLY A   1      -1.812 -11.033  14.344  1.00  0.00           H
ATOM      7  HA2 GLY A   1      -2.530 -11.286  12.736  1.00  0.00           H
ATOM      8  C   GLY A   1      -2.735  -9.310  13.501  1.00 52.75           C
ATOM      9  O   GLY A   1      -3.409  -8.826  12.583  1.00 53.51           O
ATOM     10  N   SER A   2      -2.009  -8.571  14.341  1.00 51.16           N
ATOM     11  H   SER A   2      -1.457  -9.059  15.075  1.00  0.00           H
ATOM     12  CA  SER A   2      -1.949  -7.107  14.284  1.00 50.17           C
ATOM     13  HA  SER A   2      -2.916  -6.753  13.9

In [47]:
import math

def calc_center(positions):
    center = [0.0 for _ in positions[0]]
    for pos in positions:
        for i in range(len(center)):
            center[i] += pos[i]
    for i in range(len(center)):
            center[i] /= len(positions)
    return center

def calc_distance(vec1, vec2):
    sum = 0.0
    for i in range(len(vec1)):
        sum += (vec1[i] - vec2[i])**2.0
    return math.sqrt(sum)

def calc_len(vec):
    origin = [0.0 for _ in vec]
    return calc_distance(vec, origin)

def neighbors(poi, atomset, cutoff):
    neighbors = []
    for atom in atomset:
        if calc_distance(poi, atom) <= cutoff:
            neighbors.append(atom)
    return neighbors

def diff(vec1, vec2):
    res = []
    for i in range(len(vec1)):
        res.append(vec2[i] - vec1[i])
    return res

def sum(vec1, vec2):
    res = []
    for i in range(len(vec1)):
        res.append(vec2[i] + vec1[i])
    return res

def mult(vec, scalar):
    res = []
    for i in range(len(vec)):
        res.append(vec[i] * scalar)
    return res

def norm(vec):
    l = calc_len(vec)
    return mult(vec, 1.0/l)

def calc_force(poi, neighbors, weighted=True):
    f = [0.0 for _ in neighbors[0]]
    for n in neighbors:
        vec = diff(n, poi)
        weight = 1.0
        if weighted:
            dist = calc_distance(n, poi)
            weight = 1.0/dist
        norm_vec = norm(vec)
        weighted_vec = mult(norm_vec, weight)
        f = sum(f, weighted_vec)
    return f

def update_position(pos, force, momentum, distance):
    final_force = [0.0 for _ in pos]
    final_force = sum(final_force, force)
    for m in momentum:
        final_force = sum(final_force, m)
    final_force = norm(final_force)
    final_force = mult(final_force, distance)
    return sum(pos, final_force)

for pdb in pdb_ids[:1]:

    prot_atoms = []
    lig_atoms = []

    for line in clean_pdbs[pdb].split('\n'):
        if line[0:4] == 'ATOM':
            prot_atoms.append([float(line[30:38]), float(line[38:46]), float(line[46:54])])
    mol = mols[pdb]
    conf = mol.GetConformer( 0 )
    for i in range( mol.GetNumAtoms() ):
         atom_pos = conf.GetAtomPosition( i )
         lig_atoms.append( [atom_pos.x, atom_pos.y, atom_pos.z] )

    prot_center = calc_center(prot_atoms)
    lig_center = calc_center(lig_atoms)
    states = [lig_center]
    momentum = []

    step_size = 1.0
    cutoff = 6.0
    n_states = 10

    f = calc_force(lig_center, [prot_center], weighted=False)
    states.append(update_position(lig_center, f, momentum, step_size))
    while(len(states) < n_states):
        momentum.append(diff(states[-2], states[-1]))
        nbs = neighbors(states[-1], prot_atoms, cutoff)
        f = calc_force(states[-1], nbs, weighted=True)
        states.append(update_position(lig_center, f, momentum, step_size))
    
    path_pdb = clean_pdbs[pdb]
    pdb_template = "HETATM    1  O   HOH A   1     {x:8.3f}{y:8.3f}{z:8.3f}  1.00 20.00           O  \n"
    for state in states:
        path_pdb += pdb_template.format(x=state[0], y=state[1], z=state[2])
    with open('path.pdb', 'w') as file:
        file.write(path_pdb)
    print(pdb)

3ivq


In [2]:
def test(a, b):
    c = a + b
    a = c
    print(a)

a = 5
b = 2
test(a, b)
print(a)

7
5
